In [ ]:
import json
import os
import shutil
import warnings
from datetime import datetime
import sys

import pandas as pd
import typer
from dotenv import find_dotenv, load_dotenv
from fastcore.xtras import Path

from extracao.stations import Estacoes
from extracao.datasources.srd import SRD
from extracao.format import merge_on_frequency, LIMIT_FREQ
from extracao.location import Geography


load_dotenv(find_dotenv(), override=True)
warnings.simplefilter('ignore')

SQLSERVER_PARAMS = dict(
	driver=os.environ.get('SQL_DRIVER'),
	server=os.environ.get('SQL_SERVER'),
	database=os.environ.get('SQL_DATABASE'),
	trusted_conn=True,
	mult_results=True,
	encrypt=False,
	timeout=int(os.environ.get('SQL_TIMEOUT')),
)

if sys.platform in ('linux', 'darwin', 'cygwin'):
	SQLSERVER_PARAMS.update(
		{
			'trusted_conn': False,
			'mult_results': False,
			'username': os.environ.get('USERNAME'),
			'password': os.environ.get('PASSWORD'),
		}
	)

MONGO_URI: str = os.environ.get('MONGO_URI')

In [ ]:
s = '[{"d":1},{"d":2}]'

In [ ]:
json.dumps(json.loads(s))

'[{"d": 1}, {"d": 2}]'

In [ ]:
e = data = Estacoes(SQLSERVER_PARAMS, MONGO_URI, read_cache=True, parallel=False)

In [ ]:
e.update()

  0%|                    | 0/5570 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|                   | 0/55928 [00:00<?, ?it/s]

  0%|                    | 0/5570 [00:00<?, ?it/s]

  0%|                    | 0/1668 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?it/s]

In [ ]:
e.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014960 entries, 0 to 1014959
Data columns (total 29 columns):
 #   Column                       Non-Null Count    Dtype   
---  ------                       --------------    -----   
 0   Frequência                   1014960 non-null  category
 1   Entidade                     997967 non-null   category
 2   Fistel                       1013373 non-null  category
 3   Serviço                      1013373 non-null  category
 4   Estação                      1010281 non-null  category
 5   Latitude                     1014960 non-null  category
 6   Longitude                    1014960 non-null  category
 7   Código_Município             1014799 non-null  category
 8   Município                    1014838 non-null  category
 9   UF                           1014838 non-null  category
 10  Classe                       1012945 non-null  category
 11  Classe_Emissão               1007931 non-null  category
 12  Largura_Emissão(kHz)        

In [ ]:
e.df[e.df.Código_Município.isna()]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
321,0.1143,NDB - MELO,<NA>,<NA>,<NA>,-32.340055556,-54.223888889,<NA>,,Cerro Largo-Uruguai,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
761,0.225,NDB - ITAIPU,<NA>,<NA>,<NA>,-25.40866667,-54.62166667,<NA>,,Alto Paraná-Paraguai,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
841,0.25,NDB - PASO DE LOS LIBRES,<NA>,<NA>,<NA>,-29.69472222,-57.14805556,<NA>,,Corrientes-Argentina,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1101,0.407,NDB - LETÍCIA,<NA>,<NA>,<NA>,-4.194997222,-69.939733333,<NA>,Leticia,Amazonas-Colômbia,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
35456,110.3,"ILS/DME, RIO DE JANEIRO INTL.",<NA>,<NA>,<NA>,-22.83333333333333,-43.23333333333333,<NA>,Rio de Janeiro,Rio de Janeiro,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671163,1146.0,"VOR/DME, AMAZONICA(Airbone DME)",<NA>,<NA>,<NA>,-4.183333333333334,-69.93333333333334,<NA>,Leticia,Amazonas-Colômbia,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
671288,1193.0,DME - ASUNCION 106X,<NA>,<NA>,<NA>,-25.244166667,-57.522,<NA>,Luque,Departamento Central-Paraguai,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
671318,1209.0,"VOR/DME, AMAZONICA(Ground-based DME)",<NA>,<NA>,<NA>,-4.183333333333334,-69.93333333333334,<NA>,Leticia,Amazonas-Colômbia,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
671319,1209.0,DME - LETICIA 122X,<NA>,<NA>,<NA>,-4.195,-69.940555556,<NA>,Leticia,Amazonas-Colômbia,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
e.df[e.df.UF.isna()]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
36682,126.9,"TWR 25/40, BACIA DE CAMPOS",<NA>,<NA>,<NA>,-22.357222222222223,-40.09027777777778,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
36946,130.0,"AOC U 100/100, LITORAL DO NORDESTE",<NA>,<NA>,<NA>,-19.570833333333333,-39.255,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
36948,130.0,"AOC U 100/100, BACIA DE CAMPOS",<NA>,<NA>,<NA>,-22.4375,-40.42638888888889,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
36978,130.025,"AOC U 100/100, BACIA DO ESPÍRITO SANTO",<NA>,<NA>,<NA>,-21.366666666666667,-39.95,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
36979,130.025,"AOC U 100/100, BACIA DO ESPÍRITO SANTO",<NA>,<NA>,<NA>,-22.059722222222224,-40.186388888888885,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38317,131.475,"AOC U 100/100, BACIA DE CAMPOS",<NA>,<NA>,<NA>,-22.696666666666665,-40.083333333333336,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
38318,131.475,"AOC U 100/100, RIO DAS OSTRAS",<NA>,<NA>,<NA>,-22.696666666666665,-40.085,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
38438,131.675,"AOC U 100/100, BACIA DE CAMPOS",<NA>,<NA>,<NA>,-23.0,-40.5,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
38444,131.675,"AOC U 100/100, BACIA DE CAMPOS",<NA>,<NA>,<NA>,-23.18777777777778,-40.69166666666666,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='rfdatahub')

In [ ]:
location = geolocator.reverse('-22.357222222222223, -40.09027777777778')

In [ ]:
location.raw['address']

{'man_made': 'P-20'}

In [ ]:
dfs = e.extraction()

In [ ]:
aero = dfs.pop()
anatel = pd.concat(dfs, ignore_index=True)

In [ ]:
anatel = anatel.astype('string', copy=False)

In [ ]:
anatel.info()

In [ ]:
df = merge_on_frequency(anatel, aero)


In [ ]:
df.info()

In [ ]:
g = Geography(df)


In [ ]:
df[df.Latitude.isna()]

In [ ]:
df[g.log['empty_coords']]

In [ ]:
g.log['both'].sum()

In [ ]:
g.drop_rows_without_location_info()

In [ ]:
df[g.log['empty_code']]

In [ ]:
df.columns

In [ ]:
g.df = g.merge_df_with_ibge(g.df)

In [ ]:
g.df.info()

In [ ]:
df[g.log['empty_code']].Fonte.unique()

In [ ]:
g.df[~g.log['empty_code'] & g.df['Município_IBGE'].isna()]

In [ ]:
g.normalize_location_names()

In [ ]:
folder = Path.cwd().parent / "extracao" / "datasources" / "arquivos" / "saida"

In [ ]:
def read_df(name: str)->pd.DataFrame:
    return pd.read_parquet(folder / f"{name}.parquet.gzip", dtype_backend="numpy_nullable")

In [ ]:
df = read_df("estacoes")

In [ ]:
df.info()

In [ ]:
df[df.Latitude.isna()]

In [ ]:
df.loc[df['Código_Município'].isna(), ['Frequência', 'Entidade', 'Fonte']]

In [ ]:
df.loc[671323, 'Fonte']